In [1]:
# Import block
import os
import pandas as pd
import numpy as np
import time
from math import sqrt
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pandas.plotting import scatter_matrix
from importlib import reload
from sklearn.feature_selection import VarianceThreshold
from collections import Counter
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [2]:
grenade_data_path = '../expected_data_grenade.csv'
shield_data_path = '../expected_data_shield.csv'
reload_data_path = '../expected_data_reload.csv'
logout_data_path = '../expected_data_logout.csv'
# noise_data_path = '../expected_data_noise.csv'


grenade_data_raw = pd.read_csv(grenade_data_path, index_col=False)
shield_data_raw = pd.read_csv(shield_data_path, index_col=False)
reload_data_raw = pd.read_csv(reload_data_path, index_col=False)
logout_data_raw = pd.read_csv(logout_data_path, index_col=False)
# noise_data_raw = pd.read_csv(noise_data_path, index_col=False)

grenade_data_raw = grenade_data_raw.apply(pd.to_numeric)
grenade_data_raw.reset_index(inplace=True, drop=True)

shield_data_raw = shield_data_raw.apply(pd.to_numeric)
shield_data_raw.reset_index(inplace=True, drop=True)

reload_data_raw = reload_data_raw.apply(pd.to_numeric)
reload_data_raw.reset_index(inplace=True, drop=True)

logout_data_raw = logout_data_raw.apply(pd.to_numeric)
logout_data_raw.reset_index(inplace=True, drop=True)

# noise_data_raw = noise_data_raw.drop(1491).drop(1453).apply(pd.to_numeric)
# noise_data_raw.reset_index(inplace=True, drop=True)

In [3]:
WINDOW_SIZE = 20
SLIDE_SIZE = 1

def process_df_with_sliding_window(df, window_size, slide_size):
    attributes = ["acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z"]
    columns_list = []
    # Build column list
    for idx in range(1, 1 + window_size):
        for attribute in attributes:
            columns_list.append(attribute + '_' + str(idx))
            
    df_out = pd.DataFrame(columns=columns_list)

    for row_idx in range(0, len(df) - window_size, slide_size):
        curr_window_data = []
        nan_found = False
        for row_iter_idx in range(window_size):
            curr_row_idx = row_idx + row_iter_idx
            if df.loc[curr_row_idx, :].isnull().values.any():
                nan_found = True
                break
            curr_row_list = df.loc[curr_row_idx, :].values.flatten().tolist()
            curr_window_data += curr_row_list
        
        if nan_found:
            continue

        df_out.loc[len(df_out)] = curr_window_data


    return df_out

grenade_df_processed = process_df_with_sliding_window(grenade_data_raw, WINDOW_SIZE, SLIDE_SIZE)
shield_data_processed = process_df_with_sliding_window(shield_data_raw, WINDOW_SIZE, SLIDE_SIZE)
reload_data_processed = process_df_with_sliding_window(reload_data_raw, WINDOW_SIZE, SLIDE_SIZE)
logout_data_processed = process_df_with_sliding_window(logout_data_raw, WINDOW_SIZE, SLIDE_SIZE)
# noise_data_processed = process_df_with_sliding_window(noise_data_raw, WINDOW_SIZE, SLIDE_SIZE)


In [4]:
from sklearn.preprocessing import normalize

# combined_data_features = pd.concat([grenade_df_processed, shield_data_processed, reload_data_processed, logout_data_processed, noise_data_processed])
combined_data_features = pd.concat([grenade_df_processed, shield_data_processed, reload_data_processed, logout_data_processed])
combined_data_features.reset_index(inplace=True, drop=True)
# combined_data_features = combined_data_features.apply(pd.to_numeric)

# for _, i in combined_data_features.iterrows():

#     for c in range(60):
#         if type(i[c]) != float and type(i[c]) != str:
#             print(i[c], type(i[c]))

In [5]:
# Build label
combined_data_labels = pd.DataFrame(columns=['Activity'])

for i in range(len(grenade_df_processed)):
    combined_data_labels.loc[len(combined_data_labels)] = 0
    
for i in range(len(shield_data_processed)):
    combined_data_labels.loc[len(combined_data_labels)] = 1
    
for i in range(len(reload_data_processed)):
    combined_data_labels.loc[len(combined_data_labels)] = 2
    
for i in range(len(logout_data_processed)):
    combined_data_labels.loc[len(combined_data_labels)] = 3
    
# for i in range(len(noise_data_processed)):
#     combined_data_labels.loc[len(combined_data_labels)] = 4
    
combined_data_labels

,Activity
0,0
1,0
2,0
3,0
4,0
...,...
3939,3
3940,3
3941,3
3942,3


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_data_features, combined_data_labels, test_size=0.1, random_state=42)

X_train

,acc_x_1,acc_y_1,acc_z_1,gyro_x_1,gyro_y_1,gyro_z_1,acc_x_2,acc_y_2,acc_z_2,gyro_x_2,...,acc_z_19,gyro_x_19,gyro_y_19,gyro_z_19,acc_x_20,acc_y_20,acc_z_20,gyro_x_20,gyro_y_20,gyro_z_20
2328,-53.60,25.51,25.53,-660.0,-24.0,886.0,-56.16,20.60,17.33,57.0,...,80.74,-275.0,-22.0,201.0,-4.11,-1.84,79.21,-658.0,-895.0,213.0
1235,-22.40,24.29,114.84,-579.0,-706.0,87.0,-20.85,19.88,105.60,-97.0,...,-35.20,-385.0,-253.0,368.0,24.10,-23.60,-40.81,-945.0,-82.0,456.0
1377,-25.89,3.37,141.79,-802.0,234.0,26.0,-29.61,0.79,139.90,-474.0,...,91.30,230.0,323.0,629.0,34.76,31.17,88.52,702.0,229.0,133.0
139,16.13,0.74,136.09,567.0,172.0,179.0,15.65,5.85,123.72,618.0,...,-59.82,-652.0,-335.0,500.0,31.38,-17.42,-60.23,-870.0,-961.0,967.0
811,45.91,-6.29,-91.64,853.0,612.0,6.0,31.90,-15.37,-149.08,527.0,...,-174.21,-335.0,-327.0,477.0,58.58,-8.79,-176.80,-603.0,-41.0,182.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,-67.09,60.38,71.25,-269.0,-243.0,119.0,-70.17,60.32,69.16,-228.0,...,-89.26,73.0,560.0,676.0,12.84,-53.79,-91.25,50.0,256.0,646.0
1294,-0.87,-31.52,31.75,603.0,452.0,376.0,1.31,-31.84,-45.70,956.0,...,-170.84,-601.0,-864.0,603.0,23.47,-38.07,-167.68,-873.0,-767.0,439.0
860,15.94,-9.65,-155.78,-579.0,-13.0,563.0,16.62,-12.10,-163.94,-193.0,...,-142.79,-291.0,-530.0,861.0,37.96,-35.32,-144.81,-54.0,-197.0,606.0
3507,-4.26,-24.65,169.83,-477.0,661.0,279.0,0.40,-29.64,-177.37,-102.0,...,86.38,-870.0,-441.0,-393.0,50.29,5.02,99.20,-384.0,-762.0,-811.0


In [7]:
def normalize_data(df):
    for col in df.columns:
        print("Maximum obtained: ", df[col].abs().max())
        df[col] = df[col] / df[col].abs().max()
    
    return df

X_train = normalize_data(X_train)
X_test = normalize_data(X_test)

Maximum obtained:  179.25
Maximum obtained:  178.45
Maximum obtained:  179.97
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  179.25
Maximum obtained:  178.45
Maximum obtained:  179.97
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  179.25
Maximum obtained:  175.46
Maximum obtained:  179.99
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  179.25
Maximum obtained:  168.17
Maximum obtained:  179.99
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  161.97
Maximum obtained:  159.88
Maximum obtained:  179.99
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  147.98
Maximum obtained:  159.53
Maximum obtained:  179.99
Maximum obtained:  999.0
Maximum obtained:  999.0
Maximum obtained:  998.0
Maximum obtained:  137.78
Maximum obtained:  159.53
Maximum obtained:  179.99
Maxi

In [8]:
X_train.dtypes

acc_x_1      float64
acc_y_1      float64
acc_z_1      float64
gyro_x_1     float64
gyro_y_1     float64
              ...   
acc_y_20     float64
acc_z_20     float64
gyro_x_20    float64
gyro_y_20    float64
gyro_z_20    float64
Length: 120, dtype: object

In [31]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes = (120), max_iter=180)

mlp.fit(X_train, y_train)

print(mlp.score(X_train, y_train))

mlp.score(X_test, y_test)


/Users/bryanwong/Documents/laser-tag/classifier/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1096: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1.0


/Users/bryanwong/Documents/laser-tag/classifier/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (180) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9265822784810127

In [32]:
for idx in range(len(mlp.coefs_)):
    weights = mlp.coefs_[idx].tolist()
    cpp_file = [
        'double weights', str(idx), '[', str(len(weights)), '][', str(len(weights[0])), '] = {\n'
    ]
    for row in range(len(weights)):
        for col in range(len(weights[0])):
            weights[row][col] = str(round(weights[row][col], 4))
    for index, row_weights in enumerate(weights):
        cpp_file.append('{')
        output_str = ', '.join(row_weights)
        cpp_file.append(output_str)
        append_str = "},\n"
        cpp_file.append(append_str)
    cpp_file.append('};')

    cpp_program_string = ''.join(cpp_file)
    with open('/Users/bryanwong/Documents/classifier/weights' + str(idx) + '.cpp', 'w') as f:
        f.write(cpp_program_string)
        f.close()
        
for idx in range(len(mlp.intercepts_)):
    biases = mlp.intercepts_[idx].tolist()
    cpp_file = [
        'double biases', str(idx), '[1][', str(len(biases)), '] = {'
    ]
    for col in range(len(biases)):
        biases[col] = str(round(biases[col], 4))
    output_str = ', '.join(biases)
    output_str = "{" + output_str + "}};\n"
    cpp_file.append(output_str)

    cpp_program_string = ''.join(cpp_file)
    with open('/Users/bryanwong/Documents/classifier/biases' + str(idx) + '.cpp', 'w') as f:
        f.write(cpp_program_string)
        f.close()

In [11]:
input_param = X_train.iloc[1000].tolist()
for row in range(len(input_param)):
    input_param[row] = str(round(input_param[row], 2))
output_str = ', '.join(input_param)
output_str = "test_case = [" + output_str + "]"
print(y_train.iloc[1000])
output_str


Activity    2
Name: 2765, dtype: int64


'test_case = [-0.05, 0.05, 0.25, 0.13, 0.41, 0.78, -0.04, 0.06, 0.23, 0.89, 0.82, 0.36, -0.03, 0.08, 0.21, 0.25, 0.29, 0.16, -0.02, 0.11, 0.18, 0.59, 0.84, 0.98, -0.0, 0.14, 0.14, 0.13, 0.3, 0.43, 0.02, -0.18, -0.11, 0.43, 0.76, 0.77, 0.05, -0.21, -0.07, 0.42, 0.18, 0.13, 0.1, -0.25, -0.03, 0.37, 0.33, 0.64, 0.14, -0.29, -0.02, 0.6, 0.43, 0.29, 0.23, -0.33, -0.07, 0.61, 0.29, 0.1, 0.28, -0.36, -0.13, 0.71, 0.96, 0.02, 0.35, -0.37, -0.17, 0.18, 0.55, 0.92, 0.41, -0.37, -0.2, 0.44, 0.86, 0.61, 0.44, -0.37, -0.22, 0.03, 0.13, 0.05, 0.46, -0.33, -0.22, 0.71, 0.36, 0.3, 0.47, -0.31, -0.22, -0.3, -0.25, 0.43, 0.47, -0.3, -0.21, -0.86, -0.64, 0.43, 0.47, -0.29, -0.19, -0.72, -0.2, 0.35, 0.47, -0.29, -0.18, -0.33, -0.57, 0.24, 0.46, -0.28, -0.16, -0.79, -0.9, 0.09]'

In [12]:
mlp.out_activation_

'softmax'